Data Source:

De Cock, D. (2011). Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project. Journal of Statistics Education, 19(3). https://doi.org/10.1080/10691898.2011.11889627

In [1]:
from pyreal.sample_applications import ames_housing

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In this tutorial, we use Pyreal to get feature contribution explanations for the AmesHousing dataset

First, we load in the data. Pyreal expects all data as DataFrames, where columns have the feature names.

In [2]:
x_orig, y = ames_housing.load_data(include_targets=True)
sample_houses = ames_housing.load_sample_houses()
sample_houses.set_index("Address", inplace=True)

Next, we load in the interpretable feature descriptions.

In [3]:
feature_descriptions = ames_housing.load_feature_descriptions()
feature_descriptions

{'MSSubClass': 'Identifies the type of dwelling involved in the sale.',
 'MSZoning': 'Identifies the general zoning classification of the sale.',
 'LotFrontage': 'Linear feet of street connected to property',
 'LotArea': 'Lot size in square feet',
 'Street': 'Type of road access to property',
 'Alley': 'Type of alley access to property',
 'LotShape': 'General shape of property',
 'LandContour': 'Flatness of the property',
 'Utilities': 'Type of utilities available',
 'LotConfig': 'Lot configuration',
 'LandSlope': 'Slope of property',
 'Neighborhood': 'Physical locations within Ames city limits',
 'Condition1': 'Proximity to various conditions',
 'Condition2': 'Proximity to various conditions (if more than one is present)',
 'BldgType': 'Type of dwelling',
 'HouseStyle': 'Style of dwelling',
 'OverallQual': 'Rates the overall material and finish of the house',
 'OverallCond': 'Rates the overall condition of the house',
 'YearBuilt': 'Original construction date',
 'YearRemodAdd': 'Remod

Next, we load the transfomers.

The first kind of transformer manually imputes the data based on information we know about the dataset.
We will call this the `AmesHousingImputer`

This imputation code comes from https://www.kaggle.com/juliencs/a-study-on-regression-applied-to-the-ames-dataset

In [4]:
transformers = ames_housing.load_transformers()
transformers

In [5]:
model = ames_housing.load_model()
model

Ridge()

Now, we can initialize a RealApp object.

In [6]:
from pyreal import RealApp

realApp =  RealApp(model,
                   X_train_orig=x_orig,
                   y_train=y,
                   transformers=transformers,
                   feature_descriptions=feature_descriptions,
                )

We can make predictions using RealApp objects.

In [7]:
realApp.predict(sample_houses)

{'312 Oakway Lane': 157609.44775291218,
 '3699 Jenna Lane': 208022.03968356713,
 '3880 Hazelwood Avenue': 205414.6183077359,
 '842 Jenna Lane': 208627.0181243478,
 '1580 Cameron Road': 172556.73662400094,
 '2231 Hillhaven Drive': 288442.94486161857,
 '215 Green Acres Road': 165163.16885054007,
 '2119 Edsel Road': 276234.6844264518,
 '642 Fulton Street': 232024.47875784046}

And generate different types of explanations

In [10]:
realApp.produce_feature_contributions(sample_houses.iloc[0:1])["312 Oakway Lane"]

KeyError: '312 Oakway Lane'

In [ ]:
realApp.produce_similar_examples(sample_houses.iloc[0], num_examples=2, fast=True)

In [ ]:
realApp.produce_feature_importance()